In [ ]:
import sys ###qmdp1header
sys.path.append('../scripts/')
from dp_policy_agent import *
from dynamic_programming import * #追加

In [ ]:
class QmdpAgent(DpPolicyAgent): #このセルを追加 ###qmdp1
    def __init__(self, time_interval, estimator, goal, puddles, sampling_num=10, widths=np.array([0.2, 0.2, math.pi/18]).T, \
                 puddle_coef=100.0, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T): 
        super().__init__(time_interval, estimator, goal, puddle_coef, widths, lowerleft, upperright)
        
        self.dp = DynamicProgramming(widths, goal, puddles, time_interval, sampling_num) #DPのオブジェクトを持たせる
        self.dp.value_function = self.init_value()                                                                            #ファイルから読み込んで価値関数をセット
        
    def init_value(self): #追加
        tmp = np.zeros(self.dp.index_nums)
        for line in open("value.txt", "r"):
            d = line.split()
            tmp[int(d[0]), int(d[1]), int(d[2])] = float(d[3])
            
        return tmp

In [ ]:
def trial(animation):  ###qmdp1trial（20行目）
    time_interval = 0.1
    world = PuddleWorld(30, time_interval, debug=not animation) 

    ##ランドマークの追加（意地悪な位置に）##
    m = Map()
    for ln in [(1,4), (4,1), (-4,-4)]: m.append_landmark(Landmark(*ln))
    world.append(m)        
    
    ##ゴール・水たまりの追加（これは特に変更なし）##
    goal = Goal(-3,-3)
    puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] 
    world.append(goal)
    world.append(puddles[0]) 
    world.append(puddles[1])
    
    ##ロボットを作る##
    init_pose = np.array([2.5, 2.5, 0]).T
    pf = Mcl(m, init_pose, 100)
    a = QmdpAgent(time_interval, pf, goal, puddles)                             #変更
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)
    world.draw()
    
    return a

In [ ]:
a = trial(True) #qmdp1exec
a.dp.value_function